# 9장 맵 추상 자료형 1부

Copyright (C)  Brad Miller, David Ranum. 
This work is licensed under the Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License. To view a copy of this license, visit http://creativecommons.org/licenses/by-nc-sa/4.0/.

## 목표

맵(Map) 추상 자료형의 다양한 활용법을 소개한다. 

- 해시 테이블
- 이진탐색트리
- 균형 이진탐색트리(AVL 트리)

## 9.1 `Map` 추상 자료형

사전 자료형처럼 작동하는 자료형 일반을 `Map` 추상 자료형이라 부른다.
`Map` 객체는 키(key)와 값(value)으로 이루어진 쌍을 저장하며 키(key)를
이용하여 연관된 값(value)를 확인한다.

#### `Map` 추상 자료형

`Map` 클래스는 아래 기능을 제공해야 한다.

- `put(key, val)`: 키-값 쌍을 항목으로 추가.
    키(key)가 이미 존재하는 경우 관련된 값(value)을 얻데이트함.
- `get(key)`: 키와 연관된 값 확인. 키가 존재하지 않으면 `None` 반환.
- `del`: `del map[key]` 방식으로 작동하며 지정된 키와 연관된 값을 함께 삭제함.
- `len()`: 항목 개수 반환
- `in`: 키(key) `key in map` 방식으로 작동하며 키의 존재 여부 확인.

## 9.2 해싱과 해시 테이블

**해싱**(hashing) 기법을 적용하여 $O(1)$ 시간복잡도의 탐색을 허용하는
자료구조를 알아본다.
이 자료구조는 **해시 테이블**(hash table)을 이용하여 
새로운 항목이 저장될 위치가 어디인지 미리 확인한
다음에 저장한다. 

해시 테이블의 저장 공간 각각을 **슬롯**(slot)이라 부르며
탐색 대상 항목이 있어야 하는 슬롯의 위치인 인덱스를 먼저 확인한 후에
실제로 해당 슬롯에 저장되어 있는지 여부만을 확인하기에
시간복잡도가 $O(1)$이 될 수 있다.

**주의사항**: 항목이 위치해야 할 위치를 계산하는 시간은 일정하다는 전제조건이
성립해야 한다.

여기서는 리스트를 이용하여 해시 테이블을 구현한다. 
아래 그림은 크기가 11인 비어 있는 해시 테이블을 보여준다.

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/hashtable.png" width="70%"></div>
</figure>

### 9.2.1 해시 함수

**해시 함수**(hash function)는 항목이 저정되어야 할 슬롯의 위치를 계산한다. 
길이가 $m$인 해시 테이블을 사용한다면 해시 함수의 반환값은 $0$과 $m-1$ 사이의 값을
임의의 항목에 대해 계산해야 한다.

#### 해시 알고리즘 1: 나머지 함수

입력값을 해시 테이블의 길이로 나눈 나머지를 항목이 저장되어야 할 슬롯의 위치(인덱스)로 지정한다.

```python
h(item) = item % 11
```

예를 들어 54, 26, 93, 17, 77, 31 에 대한 **해시값**(hash value)은 
다음과 같다.

| **항목** |  **해시값** |
| :---: | :---: |
| 54 | 10 |
| 26 | 4 |
| 93 | 5 |
| 17 | 6 |
| 77 | 0 |
| 31 | 9 |

해시값을 활용하여 각각의 값을 리스트의 해당 인덱스에 저장한 결과는 다음과 같다.

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/hashtable2.png" width="70%"></div>
</figure>

_**탐색 시간 복잡도**_

나머지 함수를 활용하여 생성된 해시 테이블에 대한 탐색 알고리즘은 $O(1)$이다.
이유는 항목의 나머지를 계산한 다음에 해당 위치의 값을 한 번 확인만하면 되기 때문이다.
나머지 계산에 드는 시간은 해시 테이블의 길이와 무관하게 무관하게 거의 일정하다.

#### 해시 테이블 점유율

**점유율**(load factor)은 해시 테이블에서 항목이 자리한 슬롯의 비율이며
보통 $\lambda$(람다)로 표시한다.

$$\lambda = \frac {\text{포함된 항목 수}}{\text{테이블 크기}}$$

위 해시 테이블의 점유율은 $\frac{6}{11}$이다. 

**참고**: 점유율($\lambda$)은 해싱 관련 함수들의 시간/공간 복잡도를 계산하는 데에 
중요한 역할을 수행한다(9.2.4절 참조).

#### 충돌 문제

앞서 설명한 나머지 방식은 나머지가 동일하면 저장되어야 할 
위치가 중복되는 문제를 갖는다.
이런 문제를 **충돌**(collision)이라 부른다.
예를 들어 44, 77 등은 모두 해시 테이블의 길이인 11로 나누면 나머지가 0이다. 

임의의 데이터셋에 대해 충돌이 절대로 발생하지 않는 해시 함수와 해시 테이블을 
구현하는 **완전한 해시 함수**는 존재하지 않는다. 
일반적으로 해시 테이블의 길이를 충분히 늘리면 충돌이 발생하지 않는다.
예를 들어 `2021280001` 등과 같은 10자리 학번을 충돌없이 
저장하기 위해 길이가 100억인 해시 테이블을 활용할 수 있다.
하지만 학생 수가 고작 몇 천명이라면 엄청난 양의 메모리를 낭비하게 되는 셈이다.

따라서 해시 테이블의 크기를 최소화하면서 충돌을 방지하는 해시 함수를 고안해야 하는 것이
주요 과제이다. 
여기서는 나머지 방식을 개선하는 과정을 통해 충돌이 
보다 적게 발생하는 다양한 해시 테이블 구현 기법을 알아본다.

**참고**: 나머지 계산은 어떤 형태로든 여전히 활용된다.

#### 해시 알고리즘 2: 자릿수 접기 기법

**자릿수 접기**(digit folding) 기법은 주어지는 수를 일정 크기로 쪼개서
생성된 값들에 나머지 활용 기법을 적용한다. 
단, 마지막으로 쪼개진 수의 경우 다른 수들의 자릿수와 다를 수 있다.

**방식 1**

예를 들어, 전화번호 43-655-5461 이라는 전화번호를 저장하려 할 때 두 자리씩 쪼개면
43, 65, 55, 46, 1이 생성되고,
이 수들을 다 더합면 210을 얻는다.

$$43 + 65 + 55 + 46 + 1 = 210$$

11개의 슬롯을 갖는 해시 테이블에 위 전화번호를 저장하기 위해
나머지 기법을 적용한다. 
즉, 210을 11로 나눈 나머지가 1이기에 전화번호 43-655-5461은
1번 슬롯에 저장된다. 

**방식 2**

일정 길이로 쪼갠 값들을 더해줄 때 두 개 중에 하나는 숫자를 뒤집어서 더하는 방식이다. 
이전과 동일한 전화번호를 2자로 끊어서 생성된 번호를 다음처럼 둘 중에 하나는 뒤집어서 더한 후
11로 나눈 나머지를 계산하면 다른 슬롯으로 지정된다.

$$43 + 56 + 55 + 64 + 1 = 219$$

그리고 219를 11로 나눈 나머지는 10이다. 

**방식 3**

**중간제곱법**(mid-square method)은 주어진 수를 제곱한 후 
중간에 위치한 몇 개의 숫자를 대상으로 나머지 기법을 적용한다. 
예를 들어, 44에 대해 먼저 제곱을 계산한다. 

$$44 ^{2} = 1,936$$

이제 중간에 위치한 9와 3을 추출하여 생성된 수 93에 대해 나머지 기법을 적용한다. 
해시 테이블의 크기가 11이라면 93을 11로 나눈 나머지 5가 저장될 슬롯의 인덱스가 된다. 

아래 테이블은 다양한 항목에 대해 나머지 기법과 중간제곱법을 적용한 결과가 
어떻게 다른지를 보여준다. 
단, 해시 테이블의 길이는 11이다.

| **항목** | **나머지** | **중간제곱** 
| :---: | :---: | :---: |
| 54 | 10 | 3 |
| 26 |  4 | 7 |
| 93 |  5 | 9 |
| 17 |  6 | 8 |
| 77 |  0 | 4 |
| 31 |  9 | 6 |

#### 문자열 해싱

문자열에 대한 해시 함수는 각 문자에 대응하는 정수 유니코드를 생성하는
`ord()` 함수를 이용한다. 
예를 들어, 문자열 `"cat"`에 사용된 세 개의 문자 각각에 대한 정수 유니코드는 다음과 같다.

In [1]:
print(ord("c"))
print(ord("a"))
print(ord("t"))

이제 생성된 수들에 앞서 설명한 해시 알고리즘 중에 한 가지 방식을 적용하면 된다.

**방식 1**

예를 들어, 얻어진 수를 모두 더해 해시 테이블의 길이로 나눈 나머지를 활용하는 방식을
적용하는 과정은 다음과 같다.
단, 해시 테이블의 길이는 11이라고 가정한다.

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/stringhash.png" width="45%"></div>
</figure>

아래 함수 `hash_str()` 함수가 앞서 설명한 알고리즘을 구현한다.

In [2]:
def hash_str(a_string, table_size):
    return sum([ord(c) for c in a_string]) % table_size

**방식 2**

이전 방식은 서로 어구전철(anagram)인 단어에 대해 동일한 값을 계산한다.
즉, 어구전철 단어를 구별하지 못한다. 

In [3]:
hash_str("cat", 11)

4

In [4]:
hash_str("act", 11)

4

이 단점을 해결하기 위해 각 문자의 위치 정보를 해시값을 계산할 때 함께 활용한다. 

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/stringhash2.png" width="45%"></div>
</figure>

In [5]:
def hash_str(a_string, table_size):
    return sum([ord(c)* (i+1) for i, c in enumerate(a_string)]) % table_size

In [6]:
hash_str("cat", 11)

3

In [7]:
hash_str("act", 11)

5

#### 해시 함수의 복잡도

해시값을 계산하는 알고리즘이 너무 복잡하면 별 의미가 없다.
그 대신에 리스트에 임의로 항목을 저장한 후에 순차탐색 또는 정렬 후 이진탐색 등으로
탐색을 진행하는 편이 더 나을 수 있다. 

### 9.2.2 충돌 해결법

모든 경우에 완벽한 해시 함수는 존재하지 않는다고 했다.
따라서 서로 다른 항목이 동일한 해시값을 갖는 충돌이
발생했을 때 처리 방법을 항상 고민해야 한다. 

#### 충돌 해결법 1: 빈자리 찾기

**빈자리 찾기**(open addressing)는 해시값으로 지정된 슬롯에
이미 다른 항목이 자리 잡고 있을 때 오른편으로 가장 가까운 빈자리를
**선형 탐사**(linear probing)하여 찾은 후 해당 자리에 항목을 추가하는 알고리즘이다. 
선형 탐사 과정에서 리스트의 오른편 끝에 다다른다면 
경우에 따라 리스트의 처음부터 다시 선형 탐사를 시작할 수도 있다.

아래 그림은 
54, 26, 93, 17, 77, 31을 나머지 기법으로 리스트에 추가한 결과이다. 

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/hashtable2.png" width="70%"></div>
</figure>

여기에 44, 55, 20을 빈자리 찾기 기법으로 추가한 결과는 다음과 같다.

- 44 추가하기: 11로 나눈 나머지 0
    - 0번 이미 채워젔음. 
    - 따라서 1번 인덱스에 추가.
- 55 추가하기: 11로 나눈 나머지 0. 
    - 0번, 1번이미 채워젔음. 
    - 따라서 2번 인덱스에 추가.
- 20 추가하기: 11로 나눈 나머지 9. 
    - 9번, 10번 이미 채워젔음. 
    - 리스트 처음으로 돌아감. 
    - 0번, 1번, 2번 이미 채워졌음. 
    - 따라서 3번 인덱스에 추가.

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/linearprobing1.png" width="70%"></div>
</figure>

빈자리 찾기 방식으로 채워진 해시 테이블에서 탐색하려면 항목 채우기 방식을 그대로 따라가면서 
항목을 확인하거나 빈자리가 나올 때까지 선형 탐사를 반복한다.
예를 들어, 20의 포함여부를 확인하려면 아래 과정을 수행한다.

- 11로 나눈 나머지 9 확인.
- 9번 슬롯: 20 아님
- 10번 슬롯: 20 아님
- 리스트 처음으로 돌아감.
- 0번, 1번, 2번 모두 20 아님.
- 3번 슬롯에서 20 확인.
- 결론: 20 있음.

반면에 27의 포함여부 확인과정은 다음과 같다.

- 11로 나눈 나머지 5 확인.
- 5번 슬롯: 27 아님
- 6번 슬롯: 27 아님
- 7번 슬롯: 비어 있음. 
- 결론: 27 없음.

#### 군집화와 군집화 해결책

빈자리 찾기 방식은 항목들이 해시 테이블의 한쪽에서 **군집**(cluster)을 형성할 수 있다는 것이다. 
즉, **군집화**(clustering) 문제가 발생할 수 있다.
군집화가 발생하면 일부 항목이 원래 위치해야 할 슬롯으로부터 너무 멀리 떨어져 저장되어야 하는
문제가 따라온다. 
아래 그림은 군집화로 인해 20이 엉뚱한 슬롯에 저장됨을 잘 보여준다.

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/clustering.png" width="70%"></div>
</figure>

**리해싱(rehashing)**

빈 자리 찾기를 몇 칸씩 슬롯을 건너뛰며 찾으면 군집화를 약화시킬 수 있다.
예를 들어, 아래 그림은 차례대로 44, 55, 20을 충돌이 발생하면 3칸씩 건너 뛰며 빈자리를 찾아 
추가한 결과를 보여준다.

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/linearprobing2.png" width="70%"></div>
</figure>

충돌이 발생했을 때 한 칸 또는 여러 칸을 주기적으로 건너 뛰며 빈자리를 찾는 방식을 
**리해싱**(rehashing)이라 한다. 
리해싱을 사용할 때 해시 테이블의 모든 슬롯이 방문되는 것을 보장해야 한다.
예를 들어 해시 테이블의 길이가 짝수이고 4칸씩 건너뛰면 홀 수 인덱스의 슬롯은
전혀 점유되지 않을 수도 있다.
이런 이유로 해시 테이블의 길이는 11과 같은 소수(prime number)를 사용한다.

**제곱 탐사** 

**제곱 탐사**(quadratic probing)는 충돌이 발생했을 때
건너 뛰는 슬롯의 수를 제곱수 만큼 늘려가며 찾는다.
예를 들어, 해시값 `h`에 대해 해당 슬롯이 이미 점유되었을 때
다음 빈칸은 `h+1`, `h+4`, `h+9`, `h+16`, `h+25` 등을
대상으로 차례대로 점유 여부를 확인한다.
예를 들어, 아래 그림은 차례대로 44, 55, 20을 충돌이 발생하면 
제곱 탐사 방식을 적용하여 추가한 결과를 보여준다.

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/quadratic.png" width="70%"></div>
</figure>

**체이닝** 

**체이닝**(chaining)은 충돌이 발생했을 때
다른 빈칸을 찾는 대신 해당 슬롯에 다른 항목들과 함께 모음자료형의 항목으로 저장하는 방식이다. 
아래 그림은 54, 26, 93, 17, 77, 31, 44, 55, 20을 체이닝 방식으로 저장한 결과를 보여준다.

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/chaining.png" width="70%"></div>
</figure>

해시 테이블 탐색은 먼저 해시값을 계산해서 지정된 슬롯을 확인한 다음에
포함되어 있는 모음 자료형에서 해당 항목을 탐색한다. 
체이닝 방식은 각 슬롯마다 저장되는 항목의 수를 평균적으로 훨씬 낮추기에
탐색이 그만큼 빠르게 진행되도록 한다. 

__퀴즈__

길이가 13인 해시 테이블에 27과 130을 항목으로 추가할 때 사용하는 슬롯의 인덱스는 무엇인가?

정답: 1과 0

__퀴즈__

113 , 117 , 97 , 100 , 114 , 108 , 116 , 105 , 99을
길이가 11인 해시 테이블에 빈자리 찾기 방식으로 입력한 결과를 묘사하라. 
빈칸은 밑줄 두개로 표현하며, 건너 뛰는 칸의 수는 1로 한다.

정답: `99, 100, __, 113, 114, __, 116, 117, 105, 97, 108`

### 9.2.3 `HashTable` 클래스 구현

해시 테이블(`HashTable`) 클래스를 이용하여 항목 탐색을 $O(1)$ 시간으로 실행하는 
`Map` 추상 자료형을 구현한다.
클래스에 포함되는 메서드는 다음과 같다.

**생성자**

- 키와 값을 저장하기 위해 두 개의 리스트 활용. 하나의 키-값 쌍은 동일한 인덱스에 저장됨.
    - 첫째 리스트(`slots`): 키 저장
    - 둘째 리스트(`data`): 값 저장
- 해시 테이블 길이(`size`): 11 (소수 사용)

**put()** 메서드

다음 두 함수 활용
- 해시 함수(`hash_function()`): 나머지 알고리즘
- 리해싱(`rehash()`): 선형 탐사. 1칸씩 이동하며 빈자리 찾기.

**hash_function()** 메서드

- 항목을 해시 테이블의 길이로 나눈 나머지 값 반환

**rehash()** 메서드

- 선형 탐사. 1칸씩 이동하며 빈자리 찾기

**get()** 메서드

- 처음 해시값에서 시작하여 선형 탐사 기법 적용.
- 리스트를 한 바퀴 돌아 제자리로 돌아오면 탐색 중단.

이외에 `__getitem__()`, `__setitem__()` 메서드를 
지원해서 대괄호(`[]`) 연산자를 사용하도록 해야 한다.
또한 `__in__()`과 `__len__()` 메서드를 구현해야 하지만
연습문제로 남겨둔다.

#### `HashTable` 클래스

In [8]:
class HashTable:
    def __init__(self):
        self.size = 11
        self.slots = [None] * self.size
        self.data = [None] * self.size

    def put(self, key, data):
        hash_value = self.hash_function(key, len(self.slots))

        if self.slots[hash_value] is None:
            self.slots[hash_value] = key
            self.data[hash_value] = data
        else:
            if self.slots[hash_value] == key:
                self.data[hash_value] = data  # replace
            else:
                next_slot = self.rehash(hash_value, len(self.slots))
                while (
                    self.slots[next_slot] is not None
                    and self.slots[next_slot] != key
                ):
                    next_slot = self.rehash(next_slot, len(self.slots))

                if self.slots[next_slot] is None:
                    self.slots[next_slot] = key
                    self.data[next_slot] = data
                else:
                    self.data[next_slot] = data

    def hash_function(self, key, size):
        return key % size

    def rehash(self, old_hash, size):
        return (old_hash + 1) % size

    def get(self, key):
        start_slot = self.hash_function(key, len(self.slots))

        position = start_slot
        while self.slots[position] is not None:
            if self.slots[position] == key:
                return self.data[position]
            else:
                position = self.rehash(position, len(self.slots))
                if position == start_slot:
                    return None

    def __getitem__(self, key):
        return self.get(key)

    def __setitem__(self, key, data):
        self.put(key, data)
        
    # 사전 형식으로 출력
    def __repr__(self):
        aDict = {self.slots[i]:self.data[i] for i in range(len(self.slots))}
        return aDict.__repr__()

In [9]:
aHash = HashTable()

aHash[54] = "cat"
aHash[26] = "dog"
aHash[93] = "lion"
aHash[17] = "tiger"
aHash[77] = "bird"
aHash[31] = "cow"
aHash[44] = "goat"
aHash[55] = "pig"
aHash[20] = "chicken"

In [10]:
print(aHash)

{77: 'bird', 44: 'goat', 55: 'pig', 20: 'chicken', 26: 'dog', 93: 'lion', 17: 'tiger', None: None, 31: 'cow', 54: 'cat'}


In [11]:
print(aHash[20])
print(aHash[17])

chicken
tiger


In [12]:
aHash[20] = "duck"

In [13]:
print(aHash[20])
print(aHash[99])

duck
None


#### 연습

`__in__()`과 `__len__()` 메서드를 구현하라.

### 9.2.4 해싱 분석

충돌이 없는 해시 테이블에 대한 탐색 시간은 $O(1)$이다. 
하지만 충돌이 발생하는 경우 시간 복잡도 계산은 그리 간단하지 않으며
여기서는 자세히 다루지 않는다.
다만 알려진 결과만 간단하게 정리한다. 

#### 점유율($\lambda$)과 시간복잡도

점유율($\lambda$)이 탐색 시간에 많은 영향을 미친다. 

- 점유율이 0에 가까운 경우
    - 충돌 확률 매우 낮음. 
    - $O(1)$ 시간에 가깝게 탐색 가능.
- 점유율이 1에 가까운 경우
    - 충돌 확률 매우 높음. 
    - $O(1)$ 시간 보다 느리게 탐색 진행됨.

#### 빈자리 찾기 알고리즘 시간 복잡도 

선형 탐사를 사용하는 빈자리 찾기 알고리즘에서 
충돌이 발생했을 때 실행해야 하는 비교 횟수는 다음과 같다.

- 항목이 존재하는 경우 평균 비교 횟수: $\frac{1}{2}\left(1+\frac{1}{1-\lambda}\right)$ <br>

- 항목이 존재하지 않는 경우 평균 비교 횟수: $\frac{1}{2}\left(1+\left(\frac{1}{1-\lambda}\right)^2\right)$


#### 체이닝 기법 시간 복잡도 

체이닝 기법에 사용되는 알고리즘에 따라
충돌이 발생했을 때 실행해야 하는 비교 횟수는 다르지만
기본적으로 선형 탐사 방식보다 빠르며, 
해싱은 기본적으로 체이닝 기법을 활용한다. 

**참고**: 보다 자세한 내용은 
[GeeksforGeeks - Hashing | Set 2 (Separate Chaining)](https://www.geeksforgeeks.org/hashing-set-2-separate-chaining/)를 참조한다.

**연습문제**

1. 해시 테이블의 점유도가 다음과 같을 때 평균 비교횟수를 계산한 후에 
    언제 해시 테이블의 슬롯이 부족하다고 느껴지는지를 설명하라.
    -  10% 
    -  25% 
    -  50% 
    -  75% 
    -  90% 
    -  99%
    <br><br>

1. 문자열 해싱에 각 문자의 인덱스를 가중치로 활용하는 다른 방식을 생각한 후에
    기존 방식과 비교하여 장단점을 설명하라.

## 프로그래밍 실습 과제

1. `HashTable` 클래스에 `del` 메서드를 구현하라.
    빈자리 찾기, 체이닝 등 충돌 방지에 사용된 기법에 따라 
    메서드 구현이 달라질 수 있다.
    
1. 해시 테이블의 점유율이 75%가 넘는 순간에
    해시 테이블의 크기를 두 배 정도로 늘리는 `resize()` 메서드를 구현하라.
    단, 해시 테이블의 길이는 소수가 되도록 함에 주의해야 한다.
    참고: [Hash table. Dynamic resizing](https://www.algolist.net/Data_structures/Hash_table/Dynamic_resizing)

1. 제곱 탐사 기법을 이용하여 리해싱을 구현하라.

1. `HashTable` 클래스를 체이닝 방식으로 충돌을 지원하도록 하라.
    체이닝에 AVL 트리를 활용한다.